# Load Libraries

In [ ]:
# Somehow uninstall all the nvidia libraries by pip help to make tensorflow works
# If not cuDNN error keep showing up and the code just hang
# !pip3 list | grep nvidia

In [ ]:
# !pip3 uninstall -y nvidia-cublas-cu12 --break-system-packages
# !pip3 uninstall -y nvidia-nvrtc-cu12 --break-system-packages
# !pip3 uninstall -y nvidia-runtime-cu12 --break-system-packages
# !pip3 uninstall -y nvidia-cudnn-cu12 --break-system-packages
# !pip3 uninstall -y nvidia-cufft-cu12 --break-system-packages
# !pip3 uninstall -y nvidia-curand-cu12 --break-system-packages
# !pip3 uninstall -y nvidia-cusolver-cu12 --break-system-packages
# !pip3 uninstall -y nvidia-cusparse-cu12 --break-system-packages
# !pip3 uninstall -y nvidia-nccl-cu12 --break-system-packages
# !pip3 uninstall -y nvidia-nvjitlink-cu12 --break-system-packages
# !pip3 uninstall -y nvidia-cuda-cupti-cu12 --break-system-packages
# !pip3 uninstall -y nvidia-cuda-nvrtc-cu12 --break-system-packages
# !pip3 uninstall -y nvidia-cuda-runtime-cu12 --break-system-packages

In [2]:
# !export TF_ENABLE_ONEDNN_OPTS=0

In [1]:
# IMPORTANT DO NOT DELETE
# !pip3 install nvidia-pyindex
# !pip3 install nvidia-tensorrt

In [2]:
# tensorboard                              2.17.0
# tensorboard-data-server                  0.7.2
# tensorboard-plugin-wit                   1.8.1
# tensorboardX                             2.6.2.2
# tensorflow                               2.17.0
# tensorflow-datasets                      4.9.6
# tensorflow-estimator                     2.15.0
# tensorflow-hub                           0.16.0
# tensorflow-metadata                      1.16.1
# tensorflow-probability                   0.24.0
# tensorrt                                 10.9.0.34
# tensorrt_cu12                            10.9.0.34
# tensorrt_cu12_bindings                   10.9.0.34
# tensorrt_cu12_libs                       10.9.0.34
# termcolor                                2.4.0

In [1]:
!TF_ENABLE_ONEDNN_OPTS=0
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2025-05-25 22:05:02.950458: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-25 22:05:02.971623: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-25 22:05:02.978072: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-25 22:05:02.994162: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-25 22:05:08.366392: W tensorflow/compiler/tf2

Num GPUs Available:  0


2025-05-25 22:05:14.364117: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
import warnings
warnings.filterwarnings('ignore')

import glob
import io
import datasets
import os
import time
import joblib
import json
import csv
import pathlib
import librosa
import librosa.display


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import xgboost as xgb
import seaborn as sns

from tqdm.notebook import tqdm
from PIL import Image
from matplotlib import pyplot
from collections import Counter
from pprint import pprint
from pydub import AudioSegment # sudo apt install ffmpeg
%matplotlib inline

# Preprocessing
from sklearn.preprocessing import (
    LabelEncoder, 
    StandardScaler,
    MinMaxScaler,
    scale
    )
from sklearn.model_selection import (
    GridSearchCV, 
    train_test_split, 
    RepeatedStratifiedKFold, 
    cross_val_score, 
    KFold,
    StratifiedKFold
    ) 
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    roc_curve,
    roc_auc_score, 
    precision_recall_curve,
    auc,
    precision_score, 
    recall_score, 
    f1_score
    )


import torch.utils.data
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F

from datasets import load_dataset, DatasetDict,  Audio

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, accuracy_score

from transformers import WhisperModel, WhisperFeatureExtractor, AdamW

In [3]:
df_all = pd.DataFrame()

list_dataset_name = ['coswara', 'coughvid', 'esc50', 'fsdkaggle', 'virufy']
window_length = 1

path_model_save = f'/home/l083319/Cough_Related/Results/Model_Whisper/whisper_best_model_{window_length}.pt'

for dataset_name in list_dataset_name:
    print(dataset_name)
    df = pd.read_csv(f'Dataset/Sliced_Wav/dataset_{dataset_name}_{window_length}.csv')
    df_all = pd.concat([df_all, df], axis=0)
    
df_all = df_all.reset_index(drop=True)

for col in ['prob', 'status', 'age', 'Unnamed: 0', 'gender']:
    if col in df_all.columns:
        df_all = df_all.drop([col], axis=1)
df_all['filepath'] = '/home/l083319/Cough_Related/' + df_all['filepath']

audio_df = df_all.rename(columns={
    'label': 'classID', 
    'filepath': 'full_path',
})

print(audio_df.shape)
audio_df = audio_df.sample(frac=1).groupby('classID').head(5000).reset_index(drop=True)
# audio_df = audio_df.sample(frac=1).groupby('classID').head(5000).reset_index(drop=True)
# audio_df = audio_df.sample(frac=1).groupby('classID').head(10000).reset_index(drop=True)

audio_df

coswara
coughvid
esc50
fsdkaggle
virufy
(618806, 6)


,dataset,full_path,filename,filepathslice,filenameslice,classID
0,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/6...,6afc1644-3956-49d3-89e1-d3079ab9aef2.wav,Dataset/Sliced_Wav/coughvid_1/6afc1644-3956-49...,6afc1644-3956-49d3-89e1-d3079ab9aef2.wav_1.wav,1
1,coswara,/home/l083319/Cough_Related/Dataset/Coswara-Da...,UUzQFgPDt5cC2mL6zDlgm7LvU4r2_vowel-o,Dataset/Sliced_Wav/coswara_1/UUzQFgPDt5cC2mL6z...,UUzQFgPDt5cC2mL6zDlgm7LvU4r2_vowel-o_1.wav,0
2,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/6...,6bed4542-d104-4d62-85ec-5c9455e8a4c8.wav,Dataset/Sliced_Wav/coughvid_1/6bed4542-d104-4d...,6bed4542-d104-4d62-85ec-5c9455e8a4c8.wav_7.wav,1
3,coswara,/home/l083319/Cough_Related/Dataset/Coswara-Da...,OvCRh84Tf7NC1bZT9DItq4KolF32_cough-shallow,Dataset/Sliced_Wav/coswara_1/OvCRh84Tf7NC1bZT9...,OvCRh84Tf7NC1bZT9DItq4KolF32_cough-shallow_1.wav,1
4,coswara,/home/l083319/Cough_Related/Dataset/Coswara-Da...,N7axS3aQ0TfFvTyHbqjZse8iR5T2_counting-fast,Dataset/Sliced_Wav/coswara_1/N7axS3aQ0TfFvTyHb...,N7axS3aQ0TfFvTyHbqjZse8iR5T2_counting-fast_1.wav,0
...,...,...,...,...,...,...
9995,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/a...,a46ead4b-3a1c-4815-bf90-c1774de0f3ae.wav,Dataset/Sliced_Wav/coughvid_1/a46ead4b-3a1c-48...,a46ead4b-3a1c-4815-bf90-c1774de0f3ae.wav_0.wav,1
9996,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/b...,b4312be7-d7ff-49a8-93c0-379eda109f1c.wav,Dataset/Sliced_Wav/coughvid_1/b4312be7-d7ff-49...,b4312be7-d7ff-49a8-93c0-379eda109f1c.wav_5.wav,1
9997,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/0...,0f36d1fe-9013-4f43-b45f-84b460381f6a.wav,Dataset/Sliced_Wav/coughvid_1/0f36d1fe-9013-4f...,0f36d1fe-9013-4f43-b45f-84b460381f6a.wav_8.wav,1
9998,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/7...,76633e81-84f4-4c15-b374-eab1dae40188.wav,Dataset/Sliced_Wav/coughvid_1/76633e81-84f4-4c...,76633e81-84f4-4c15-b374-eab1dae40188.wav_0.wav,1


In [4]:
Counter(audio_df['dataset'])

Counter({'coughvid': 5294,
         'coswara': 3572,
         'fsdkaggle': 1002,
         'esc50': 123,
         'virufy': 9})

In [5]:
Counter(audio_df['classID'])

Counter({1: 5000, 0: 5000})

In [6]:
train_df, temp_df = train_test_split(audio_df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# train_df, temp_df = train_test_split(audio_df, test_size=0.85, random_state=42)
# val_df, test_df = train_test_split(temp_df, test_size=0.98, random_state=42)

print('Train:', len(train_df))
print('Val  :', len(val_df))
print('Test :', len(test_df))

Train: 7000
Val  : 1500
Test : 1500


In [7]:
test_df

,dataset,full_path,filename,filepathslice,filenameslice,classID
2697,coswara,/home/l083319/Cough_Related/Dataset/Coswara-Da...,LJ7VSEWnDqb3lPknPM9Q64zjdof1_cough-heavy,Dataset/Sliced_Wav/coswara_1/LJ7VSEWnDqb3lPknP...,LJ7VSEWnDqb3lPknPM9Q64zjdof1_cough-heavy_7.wav,1
6871,coswara,/home/l083319/Cough_Related/Dataset/Coswara-Da...,LOeA84grAdO6mYfDRd9fqFEICZ03_vowel-a,Dataset/Sliced_Wav/coswara_1/LOeA84grAdO6mYfDR...,LOeA84grAdO6mYfDRd9fqFEICZ03_vowel-a_4.wav,0
3487,coswara,/home/l083319/Cough_Related/Dataset/Coswara-Da...,hte8VptUoGVFEqvHpbh5brgfcNP2_counting-normal,Dataset/Sliced_Wav/coswara_1/hte8VptUoGVFEqvHp...,hte8VptUoGVFEqvHpbh5brgfcNP2_counting-normal_7...,0
92,fsdkaggle,/home/l083319/Cough_Related/Dataset/FSDKaggle2...,01bb344f,Dataset/Sliced_Wav/fsdkaggle_1/01bb344f_1.wav,01bb344f_1.wav,0
9537,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/d...,d13a37e6-981d-435d-a030-33d738c7fb20.wav,Dataset/Sliced_Wav/coughvid_1/d13a37e6-981d-43...,d13a37e6-981d-435d-a030-33d738c7fb20.wav_2.wav,1
...,...,...,...,...,...,...
9974,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/2...,2d3798dc-51ab-4e80-8fca-e5e64b8d267e.wav,Dataset/Sliced_Wav/coughvid_1/2d3798dc-51ab-4e...,2d3798dc-51ab-4e80-8fca-e5e64b8d267e.wav_8.wav,1
7025,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/7...,727c1418-7f6f-441b-ba25-f75794218373.wav,Dataset/Sliced_Wav/coughvid_1/727c1418-7f6f-44...,727c1418-7f6f-441b-ba25-f75794218373.wav_0.wav,0
3542,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/0...,0e56a1e9-7c2a-4df8-bddf-5f27bcd761ff.wav,Dataset/Sliced_Wav/coughvid_1/0e56a1e9-7c2a-4d...,0e56a1e9-7c2a-4df8-bddf-5f27bcd761ff.wav_9.wav,0
5865,fsdkaggle,/home/l083319/Cough_Related/Dataset/FSDKaggle2...,1bdc4940,Dataset/Sliced_Wav/fsdkaggle_1/1bdc4940_7.wav,1bdc4940_7.wav,0


In [8]:
train_audio_dataset = datasets.Dataset.from_dict({
    "audio": train_df["full_path"].tolist(),
    "labels": train_df["classID"].tolist()    
    }).cast_column("audio", Audio(sampling_rate=16_000))

test_audio_dataset = datasets.Dataset.from_dict({
    "audio": test_df["full_path"].tolist(),
    "labels": test_df["classID"].tolist()
    }).cast_column("audio", Audio(sampling_rate=16_000))

val_audio_dataset = datasets.Dataset.from_dict({
    "audio": val_df["full_path"].tolist(),
    "labels": val_df["classID"].tolist()
    }).cast_column("audio", Audio(sampling_rate=16_000))

In [9]:
model_checkpoint = "openai/whisper-base"

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_checkpoint)
encoder = WhisperModel.from_pretrained(model_checkpoint)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
class SpeechClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, audio_data,  text_processor):
        self.audio_data = audio_data
        self.text_processor = text_processor

    def __len__(self):
        return len(self.audio_data)

    def __getitem__(self, index):

        inputs = self.text_processor(
            self.audio_data[index]["audio"]["array"],
            return_tensors="pt",
            sampling_rate=self.audio_data[index]["audio"]["sampling_rate"]
        )

        input_features = inputs.input_features
        decoder_input_ids = torch.tensor([[1, 1]]) * encoder.config.decoder_start_token_id

        labels = np.array(self.audio_data[index]['labels'])

        return input_features, decoder_input_ids, torch.tensor(labels)


In [11]:
train_dataset = SpeechClassificationDataset(train_audio_dataset, feature_extractor)
test_dataset = SpeechClassificationDataset(test_audio_dataset, feature_extractor)
val_dataset = SpeechClassificationDataset(val_audio_dataset, feature_extractor)

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [12]:
class SpeechClassifier(nn.Module):
    def __init__(self, num_labels, encoder):
        super(SpeechClassifier, self).__init__()
        self.encoder = encoder
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.config.hidden_size, 4096),
            nn.ReLU(),
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, num_labels)
        )

    def forward(self, input_features, decoder_input_ids):
        outputs = self.encoder(input_features, decoder_input_ids=decoder_input_ids)
        pooled_output = outputs['last_hidden_state'][:, 0, :]
        logits = self.classifier(pooled_output)
        return logits

In [13]:
num_labels = 2

model = SpeechClassifier(num_labels, encoder).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, betas=(0.9, 0.999), eps=1e-08)
criterion = nn.CrossEntropyLoss()

In [14]:
# Define the training function
def train(model, train_loader, val_loader, optimizer,  criterion, device, num_epochs):

    best_accuracy = 0.0

    for epoch in range(num_epochs):
        
        model.train()

        start = time.time()
        for i, batch in enumerate(train_loader):
            
            input_features, decoder_input_ids, labels = batch

            input_features = input_features.squeeze()
            input_features = input_features.to(device)

            decoder_input_ids = decoder_input_ids.squeeze()
            decoder_input_ids = decoder_input_ids.to(device)

            labels = labels.view(-1)
            labels = labels.to(device)

            optimizer.zero_grad()

            logits = model(input_features, decoder_input_ids)

            loss = criterion(logits, labels)
            loss.backward()

            optimizer.step()
                    
            # if (i+1) % 10 == 0:
            if True:
                end = time.time()
                print(f'Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_loader)}, Train Loss: {loss.item() :.4f}, Run-time: {round(end - start, 3)}s')
                train_loss = 0.0
                start = time.time()

        val_loss, val_accuracy, val_f1, _ , _, _ = evaluate(model, val_loader, device)

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), path_model_save)

        print("========================================================================================")
        print(f'Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1: {val_f1:.4f}, Best Accuracy: {best_accuracy:.4f}')
        print("========================================================================================")

In [15]:
def evaluate(model, data_loader,  device):

    all_labels = []
    all_preds = []
    all_probs = []
    total_loss = 0.0

    with torch.no_grad():

        # for i, batch in tqdm(enumerate(data_loader), desc="Evaluating"):
        for i, batch in tqdm(enumerate(data_loader), total=len(data_loader), desc="Evaluating"):

            input_features, decoder_input_ids, labels = batch

            input_features = input_features.squeeze()
            input_features = input_features.to(device)

            decoder_input_ids = decoder_input_ids.squeeze()
            decoder_input_ids = decoder_input_ids.to(device)

            labels = labels.view(-1)
            labels = labels.to(device)

            optimizer.zero_grad()

            logits = model(input_features, decoder_input_ids)

            loss = criterion(logits, labels)
            total_loss += loss.item()

            probs = F.softmax(logits, dim=1)
            _, preds = torch.max(logits, 1)
            
            all_labels.append(labels.cpu().numpy())
            all_preds.append(preds.cpu().numpy())
            all_probs.append(probs.cpu().numpy())

    all_labels = np.concatenate(all_labels, axis=0)
    all_preds = np.concatenate(all_preds, axis=0)
    all_probs = np.concatenate(all_probs, axis=0)

    loss = total_loss / len(data_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    return loss, accuracy, f1, all_labels, all_preds, all_probs

In [16]:
num_epochs = 1

# state_dict = torch.load('/home/l083319/Cough_Related/Results/Model/whisper_best_model.pt')
# encoder = WhisperModel.from_pretrained(model_checkpoint)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# num_labels = 2
# model = SpeechClassifier(num_labels, encoder).to(device)
# model.load_state_dict(state_dict)
start = time.time() 
train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs)
end = time.time() 

print(f"Total runtime of the program is {round(end - start, 3)}s") 
print('Training Done!')

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1/1, Batch 1/219, Train Loss: 0.7055, Run-time: 105.867s
Epoch 1/1, Batch 2/219, Train Loss: 0.6396, Run-time: 101.072s
Epoch 1/1, Batch 3/219, Train Loss: 0.6114, Run-time: 98.15s
Epoch 1/1, Batch 4/219, Train Loss: 0.7040, Run-time: 96.984s
Epoch 1/1, Batch 5/219, Train Loss: 0.6599, Run-time: 95.165s
Epoch 1/1, Batch 6/219, Train Loss: 0.6136, Run-time: 94.86s
Epoch 1/1, Batch 7/219, Train Loss: 0.4825, Run-time: 97.441s
Epoch 1/1, Batch 8/219, Train Loss: 0.3829, Run-time: 95.065s
Epoch 1/1, Batch 9/219, Train Loss: 0.4306, Run-time: 99.201s
Epoch 1/1, Batch 10/219, Train Loss: 0.3764, Run-time: 96.857s
Epoch 1/1, Batch 11/219, Train Loss: 0.2423, Run-time: 99.227s
Epoch 1/1, Batch 12/219, Train Loss: 0.2972, Run-time: 99.283s
Epoch 1/1, Batch 13/219, Train Loss: 0.4103, Run-time: 96.9s
Epoch 1/1, Batch 14/219, Train Loss: 0.1557, Run-time: 100.092s
Epoch 1/1, Batch 15/219, Train Loss: 0.1056, Run-time: 99.053s
Epoch 1/1, Batch 16/219, Train Loss: 0.1493, Run-time: 99.953s
Ep

Evaluating:   0%|          | 0/47 [00:00<?, ?it/s]

Epoch 1/1, Val Loss: 0.1580, Val Accuracy: 0.9380, Val F1: 0.9380, Best Accuracy: 0.9380
Total runtime of the program is 14351.414s
Training Done!


In [17]:
print('Load Whisper Model')

# path_model_save = f'/home/l083319/Cough_Related/Results/Model_Whisper/whisper_best_model_10_5000.pt'
# path_model_save = f'/home/l083319/Cough_Related/Results/Model_Whisper/whisper_best_model_5_5000.pt'

# state_dict = torch.load('best_model.pt')
state_dict = torch.load(path_model_save)

# Create a new instance of the model and load the state dictionary
num_labels = 2
model = SpeechClassifier(num_labels, encoder).to(device)
model.load_state_dict(state_dict)

print('Evaluate Data')
_, _, _, all_labels, all_preds, all_probs = evaluate(model, test_loader, device)

print(classification_report(all_labels, all_preds))
print(accuracy_score(all_labels, all_preds))
print('Training Done!')

Load Whisper Model
Evaluate Data


Evaluating:   0%|          | 0/47 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.97      0.92      0.95       752
           1       0.93      0.98      0.95       748

    accuracy                           0.95      1500
   macro avg       0.95      0.95      0.95      1500
weighted avg       0.95      0.95      0.95      1500

0.95
Training Done!


In [18]:
y_test = all_labels
y_predict = all_preds

acc = accuracy_score(y_test, y_predict)
cm = confusion_matrix(y_test, y_predict)

lr_fpr, lr_tpr, _ = roc_curve(y_test, all_probs[:,1])
roc_auc = auc(lr_fpr, lr_tpr)
precision, recall, _ = precision_recall_curve(y_test, all_probs[:,1])
pr_auc = auc(recall, precision)

pre = precision_score(y_test, y_predict)
rec = recall_score(y_test, y_predict)
f1 = f1_score(y_test, y_predict)
tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
spe = tn / (tn + fp)
sen = rec

columns = ['dataset', 'label_count', 'window_length',
           'model',
           'acc', 'sen', 'spe', 'pre', 'rec', 'f1', 'auc', 'auprc', 'cm']  

results = [[
    Counter(audio_df['dataset']),
    Counter(audio_df['classID']),
    window_length,
    'Whisper',
    acc,
    sen,
    spe,
    pre,
    rec,
    f1,
    roc_auc,
    pr_auc,
    cm]]

df_results = pd.DataFrame(results, columns=columns)
df_results.to_csv(f'/home/l083319/Cough_Related/Results/Model_Whisper/results_summary_{window_length}.csv', index=False)

In [19]:
df_results

,dataset,label_count,window_length,model,acc,sen,spe,pre,rec,f1,auc,auprc,cm
0,"{'coughvid': 5294, 'coswara': 3572, 'fsdkaggle...","{1: 5000, 0: 5000}",1,Whisper,0.95,0.975936,0.924202,0.927573,0.975936,0.95114,0.991801,0.99186,"[[695, 57], [18, 730]]"


In [20]:
test_df['pred'] = all_preds
test_df.to_csv(f'/home/l083319/Cough_Related/Results/Model_Whisper/results_test_data_{window_length}.csv', index=False)
test_df

,dataset,full_path,filename,filepathslice,filenameslice,classID,pred
2697,coswara,/home/l083319/Cough_Related/Dataset/Coswara-Da...,LJ7VSEWnDqb3lPknPM9Q64zjdof1_cough-heavy,Dataset/Sliced_Wav/coswara_1/LJ7VSEWnDqb3lPknP...,LJ7VSEWnDqb3lPknPM9Q64zjdof1_cough-heavy_7.wav,1,1
6871,coswara,/home/l083319/Cough_Related/Dataset/Coswara-Da...,LOeA84grAdO6mYfDRd9fqFEICZ03_vowel-a,Dataset/Sliced_Wav/coswara_1/LOeA84grAdO6mYfDR...,LOeA84grAdO6mYfDRd9fqFEICZ03_vowel-a_4.wav,0,0
3487,coswara,/home/l083319/Cough_Related/Dataset/Coswara-Da...,hte8VptUoGVFEqvHpbh5brgfcNP2_counting-normal,Dataset/Sliced_Wav/coswara_1/hte8VptUoGVFEqvHp...,hte8VptUoGVFEqvHpbh5brgfcNP2_counting-normal_7...,0,0
92,fsdkaggle,/home/l083319/Cough_Related/Dataset/FSDKaggle2...,01bb344f,Dataset/Sliced_Wav/fsdkaggle_1/01bb344f_1.wav,01bb344f_1.wav,0,0
9537,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/d...,d13a37e6-981d-435d-a030-33d738c7fb20.wav,Dataset/Sliced_Wav/coughvid_1/d13a37e6-981d-43...,d13a37e6-981d-435d-a030-33d738c7fb20.wav_2.wav,1,1
...,...,...,...,...,...,...,...
9974,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/2...,2d3798dc-51ab-4e80-8fca-e5e64b8d267e.wav,Dataset/Sliced_Wav/coughvid_1/2d3798dc-51ab-4e...,2d3798dc-51ab-4e80-8fca-e5e64b8d267e.wav_8.wav,1,1
7025,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/7...,727c1418-7f6f-441b-ba25-f75794218373.wav,Dataset/Sliced_Wav/coughvid_1/727c1418-7f6f-44...,727c1418-7f6f-441b-ba25-f75794218373.wav_0.wav,0,0
3542,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/0...,0e56a1e9-7c2a-4df8-bddf-5f27bcd761ff.wav,Dataset/Sliced_Wav/coughvid_1/0e56a1e9-7c2a-4d...,0e56a1e9-7c2a-4df8-bddf-5f27bcd761ff.wav_9.wav,0,1
5865,fsdkaggle,/home/l083319/Cough_Related/Dataset/FSDKaggle2...,1bdc4940,Dataset/Sliced_Wav/fsdkaggle_1/1bdc4940_7.wav,1bdc4940_7.wav,0,0


In [21]:
# Check which data is predicted wrongly
test_df_wrong = test_df[test_df['classID'] != test_df['pred']]
test_df_wrong

,dataset,full_path,filename,filepathslice,filenameslice,classID,pred
37,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/6...,6f605ea8-96a9-4034-aeba-c85b52bcc2ea.wav,Dataset/Sliced_Wav/coughvid_1/6f605ea8-96a9-40...,6f605ea8-96a9-4034-aeba-c85b52bcc2ea.wav_2.wav,0,1
107,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/3...,31305525-0ab3-43b3-aff0-33d979714a47.wav,Dataset/Sliced_Wav/coughvid_1/31305525-0ab3-43...,31305525-0ab3-43b3-aff0-33d979714a47.wav_3.wav,0,1
9419,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/5...,53325adb-0ff2-480a-8af0-fd9da2a10322.wav,Dataset/Sliced_Wav/coughvid_1/53325adb-0ff2-48...,53325adb-0ff2-480a-8af0-fd9da2a10322.wav_3.wav,1,0
99,coswara,/home/l083319/Cough_Related/Dataset/Coswara-Da...,EyzoLFEmr5dK0ifWxP8HdCypw4r1_breathing-deep,Dataset/Sliced_Wav/coswara_1/EyzoLFEmr5dK0ifWx...,EyzoLFEmr5dK0ifWxP8HdCypw4r1_breathing-deep_9.wav,0,1
4112,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/4...,41446df9-b07a-48f2-8af0-8c73ab7a8322.wav,Dataset/Sliced_Wav/coughvid_1/41446df9-b07a-48...,41446df9-b07a-48f2-8af0-8c73ab7a8322.wav_3.wav,0,1
...,...,...,...,...,...,...,...
5156,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/4...,4731c69c-6a3d-414b-a786-43bede324991.wav,Dataset/Sliced_Wav/coughvid_1/4731c69c-6a3d-41...,4731c69c-6a3d-414b-a786-43bede324991.wav_4.wav,1,0
7655,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/2...,24a12311-7fa8-4298-9e9b-4615e00d816a.wav,Dataset/Sliced_Wav/coughvid_1/24a12311-7fa8-42...,24a12311-7fa8-4298-9e9b-4615e00d816a.wav_7.wav,0,1
2232,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/3...,3473e350-66f3-45e6-9761-ad3594d2f49f.wav,Dataset/Sliced_Wav/coughvid_1/3473e350-66f3-45...,3473e350-66f3-45e6-9761-ad3594d2f49f.wav_2.wav,0,1
6856,coughvid,/home/l083319/Cough_Related/Dataset/coughvid/3...,3f71d4e4-f3d5-4183-a9c7-d9a2c0db550b.wav,Dataset/Sliced_Wav/coughvid_1/3f71d4e4-f3d5-41...,3f71d4e4-f3d5-4183-a9c7-d9a2c0db550b.wav_3.wav,0,1


In [22]:
Counter(test_df_wrong['dataset'])

Counter({'coughvid': 58, 'coswara': 13, 'fsdkaggle': 3, 'esc50': 1})